<a href="https://colab.research.google.com/github/fadillahnuzul/Content-based-Recommender-System/blob/master/coba_skripsi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Import library**

In [ ]:
from google.colab import drive
import pandas as pd
import seaborn as sns
import numpy as np
from numpy import dot
from numpy.linalg import norm
from scipy.spatial import distance
from tabulate import tabulate

#untuk membuka dialog :
import tkinter as tk
from tkinter import filedialog, Text
import os

: 

In [ ]:
drive.mount('/content/drive')

: 


##**Preprocessing Data**
Menghilangkan missing value dan kolom serat

In [ ]:
def preprocessing(df) :
  #menghapus kolom serat
  df.drop(['serat'], axis = 1, inplace = True)
  #mendeteksi dan menghapus missing value
  print("Sebelum missing value dihilangkan :\n",df.isnull().sum())
  print("Ukuran data sebelum missing value dihilangkan :", df.shape)
  df = df.dropna(axis=0)
  print("Setelah missing value dihilangkan :\n",df.isnull().sum())
  df.reset_index(drop=True, inplace = True)
  print("Ukuran data setelah missing value dihilangkan :", df.shape)
  df.head()
  return df
# #mendeteksi dan menghilangkan outlier
# sns.boxplot(df['kalori']).set(title='Deteksi Outlier Kalori')
# Q1 = np.percentile(df['kalori'], 25,
#                    interpolation = 'midpoint')
 
# Q3 = np.percentile(df['kalori'], 75,
#                    interpolation = 'midpoint')
# IQR = Q3 - Q1
 
# print("Sebelum menghilangkan outlier: ", df.shape)
 
# # Upper bound
# upper = np.where(df['kalori'] >= (Q3+1.5*IQR))
# # Lower bound
# lower = np.where(df['kalori'] <= (Q1-1.5*IQR))
 
# ''' Removing the Outliers '''
# df.drop(upper[0], inplace = True)
# df.drop(lower[0], inplace = True)
# df.reset_index(inplace = True)
# print("Setelah menghilangkan outlier: ", df.shape)

: 

## **Perhitungan BMR**
Menghitung kebutuhan energi (kalori), karbohidrat, protein, dan lemak

In [ ]:
def perhitungan_gizi_harian() :
  # window = tk.Tk()
  # window.mainloop()
  #Input data
  tinggi = int(input("Input tinggi badan (dalam cm) : "))
  tinggi = tinggi/100
  berat = int(input("Input berat badan (dalam kg) : "))
  usia = int(input("Input usia (dalam tahun) : "))
  jenis_kelamin = input("Input jenis kelamin (L/P) : ")
  jenis_kerja = input("Jenis pekerjaan \n1.Ringan (75% duduk/berdiri, 25% berdiri/bergerak) \n2.Sedang (25% duduk/berdiri, 75% berdiri/bergerak) \n3.Berat (40% duduk/berdiri, 60% aktivitas) \nInput jenis pekerjaan (1/2/3) : ")

  #Perhitungan energi harian
  if jenis_kelamin == 'L' or  jenis_kelamin == 'l' :
    BMR = 66.473 + (13.752*berat) + (5.003*tinggi) - (6.755*usia)
    if jenis_kerja == '1' :
      energi = BMR*1.56
    elif jenis_kerja == '2' :
      energi = BMR*1.76
    elif jenis_kerja == '3' :
      energi = BMR*2.1
  elif jenis_kelamin == 'P' or  jenis_kelamin == 'p' :
    BMR = 655.096 + (9.563*berat) + (1.850*tinggi) - (4.676*usia)
    if jenis_kerja == '1' :
      energi = BMR*1.55
    elif jenis_kerja == '2' :
      energi = BMR*1.7
    elif jenis_kerja == '3' :
      energi = BMR*2

  #Menghitung BMI
  bmi = berat/(tinggi**2)
  if bmi < 17 :
    bmi_kategori = "Kurus (kekurangan berat badan tingkat berat)"
    energi = energi + 500
  elif 17 <= bmi < 18.5 :
    bmi_kategori = "Kurus (kekurangan berat badan tingkat ringan)"
    energi = energi + 500
  elif 18.5 <= bmi <= 25 :
    bmi_kategori = "Normal"
  elif 25 < bmi <= 27 :
    bmi_kategori = "Gemuk (kelebihan berat badan tingkat ringan)"
    energi = energi - 500
  elif bmi > 27 :
    bmi_kategori = "Gemuk (kelebihan berat badan tingkat berat)"
    energi = energi - 500
  print(bmi_kategori)
  energi = round(energi, 3)
  print(energi, "kalori")
  #perhitungan protein, karbohidrat, lemak
  protein = round((12/100*energi)/4, 3)
  karbo = round((68/100*energi)/4, 3)
  lemak = round((20/100*energi)/9, 3)
  #mencari kebutuhan nutrisi tiap kali makan, dikurangi dengan jumlah gizi nasi
  # energi = energi/3 - 180   #estimasi 3x makan
  # protein = protein/3 - 3
  # karbo = karbo/3 - 39.8
  # lemak = lemak/3 - 0.3
  array_gizi = [energi, protein, karbo, lemak] #kalori dibuat perhari
  return (array_gizi)

: 

In [ ]:
array_gizi = perhitungan_gizi_harian()

: 

### **Pengurangan dengan Nutrisi Nasi, Sayuran B, dan Buah**

In [ ]:
#pengurangan nutrisi nasi (180 kal, 3 gr protein, 39.8 gr karbo, 0.3 lemak)
def nutrisi_nasi(df) :
  df[0] = df[0] - 180
  df[1] = df[1] - 3
  df[2] = df[2] - 39.8
  df[3] = df[3] - 0.3
  return df

#pengurangan nutrisi sayuran B (25 kal, 1 gr protein, 5 gr karbo)
def nutrisi_sayur(df) :
  df[0] = df[0] - 25
  df[1] = df[1] - 1
  df[2] = df[2] - 5
  return df

#mengambil buah dan sayur secara acak
def random_buah_sayur(df) :
  df.sample()
  return df

#pengurangan nutrisi buah (50 kal, 12 gr karbo)
def nutrisi_buah(df_nutrisi, df_buah) :
  df_buah = df_buah.to_numpy()
  df_nutrisi[0] = df_nutrisi[0] - df_buah[0][1]
  df_nutrisi[1] = df_nutrisi[1] - df_buah[0][2]
  df_nutrisi[2] = df_nutrisi[2] - df_buah[0][3]
  df_nutrisi[3] = df_nutrisi[3] - df_buah[0][4]
  return df_nutrisi

: 

## **Colaborative Filtering**

In [ ]:
def colab_filtering(df, gizi) :
  #euclidean distance
  a = gizi
  df['euclidean'] = np.nan
  for i in range (len(df)) :
    b = [df['kalori'][i], df['protein'][i], df['karbo'][i], df['lemak'][i]]
    b = b / df['sajian_per_resep'][i]
    df['euclidean'][i] = distance.euclidean(a,b)
  df = df.sort_values(['euclidean'], ascending=[True])
  #Pembobotan nutrisi dan rating
  n = 0.5
  max_jarak = max(df['euclidean'])
  for i in df :
    jarak = df['euclidean']
    rating = df['rating']
    df['score'] = n*(rating/5) + (1-n)*(1-(jarak/max_jarak))
  menu_rekomendasi = df.sort_values(['score'], ascending=[False])
  return menu_rekomendasi

: 

## **Menampilkan Rekomendasi**

In [ ]:
#menampilkan rekomendasi menu
def tampil_rekomendasi(rekomendasi, resep_bahan) :
  data_resep_bahan = pd.read_excel("drive/My Drive/Skripsi/DataUmami.xlsx", sheet_name=resep_bahan)
  inner_join = pd.merge(rekomendasi, data_resep_bahan, on ='nama_resep', how ='inner')
  return (inner_join[['nama_resep','bahan_bahan','cara_membuat']])

: 

# **MAIN FUNGSI**

In [ ]:
#Membaca data
df_sarapan = pd.read_excel("drive/My Drive/Skripsi/DataUmami.xlsx", sheet_name='sarapan')
df_sarapan = pd.DataFrame(df_sarapan)
df_siang = pd.read_excel("drive/My Drive/Skripsi/DataUmami.xlsx", sheet_name='makan_siang')
df_siang = pd.DataFrame(df_siang)
df_malam = pd.read_excel("drive/My Drive/Skripsi/DataUmami.xlsx", sheet_name='makan_malam')
df_malam = pd.DataFrame(df_malam)
df_camilan = pd.read_excel("drive/My Drive/Skripsi/DataUmami.xlsx", sheet_name='camilan')
df_camilan = pd.DataFrame(df_camilan)
df_buah = pd.read_excel("drive/My Drive/Skripsi/DataBahanPenukar.xlsx", sheet_name='buah')
df_buah = pd.DataFrame(df_buah)
df_sayurA = pd.read_excel("drive/My Drive/Skripsi/DataBahanPenukar.xlsx", sheet_name='sayurA')
df_sayurA = pd.DataFrame(df_sayurA)
df_sayurB = pd.read_excel("drive/My Drive/Skripsi/DataBahanPenukar.xlsx", sheet_name='sayurB')
df_sayurB = pd.DataFrame(df_sayurB)

#Preprocessing data
df_sarapan = preprocessing(df_sarapan)
df_siang = preprocessing(df_siang)
df_malam = preprocessing(df_malam)
df_camilan = preprocessing(df_camilan)

#Menentukan kebutuhan gizi pengguna
array_gizi = perhitungan_gizi_harian()

#Pembagian gizi harian
sarapan, siang, malam, camilan = [], [], [], []
for i in range (len(array_gizi)) :
  sarapan.append(round(20/100*array_gizi[i], 3))
  siang.append(round(25/100*array_gizi[i], 3))
  malam.append(round(25/100*array_gizi[i], 3))
  camilan.append(round(10/100*array_gizi[i],3))

#Mengurangi nutrisi dengan nutrisi nasi pada menu makanan utama
sarapan = nutrisi_nasi(sarapan)
siang = nutrisi_nasi(siang)
malam = nutrisi_nasi(malam)

#Mengurangi nutrisi dengan nutrisi sayuran B pada menu makanan utama
sayur_pagi = df_sayurA.sample(1)
sayurA_siang = df_sayurA.sample(1)
sayurA_malam = df_sayurA.sample(1)
sayurB_siang = df_sayurB.sample(1)
sayurB_malam = df_sayurB.sample(1)
siang = nutrisi_sayur(siang)
malam = nutrisi_sayur(malam)

#Mengurangi nutrisi dengan nutrisi buah pada menu makanan utama
buah_siang = df_buah.sample(1)
buah_malam = df_buah.sample(1)
siang = nutrisi_buah(siang, buah_siang)
malam = nutrisi_buah(malam, buah_siang)

#Menentukan rekomendasi
rekomendasi_sarapan = colab_filtering(df_sarapan,sarapan).head(1)
rekomendasi_siang = colab_filtering(df_siang,siang).head(1)
rekomendasi_malam = colab_filtering(df_malam,malam).head(1)
rekomendasi_camilan = colab_filtering(df_camilan,camilan).head(3)

#Menampilkan rekomendasi
tampil_sarapan = tampil_rekomendasi(rekomendasi_sarapan, 'sarapan_resep_bahan')
tampil_siang = tampil_rekomendasi(rekomendasi_siang, 'makan_siang_resep_bahan')
tampil_malam = tampil_rekomendasi(rekomendasi_malam, 'makan_malam_resep_bahan')
tampil_camilan = tampil_rekomendasi(rekomendasi_camilan, 'camilan_resep_bahan')

pd.set_option('max_rows', 99999)
pd.set_option('max_colwidth', 100)
pd.describe_option('max_colwidth')
print("\n\n\n################# SARAPAN ####################")
print(tabulate(tampil_sarapan, headers='keys', tablefmt='pretty'))
print("sayuran pagi:\n",sayur_pagi.nama_sayur.to_string(index=False))
print("\n\n\n################# CAMILAN PAGI ####################")
print(tabulate(tampil_camilan.loc[[0]], headers='keys', tablefmt='pretty'))
print("\n\n\n################# MAKAN SIANG ####################")
print(tabulate(tampil_siang, headers='keys', tablefmt='pretty'))
print("sayuran siang:\n", sayurA_siang.nama_sayur.to_string(index=False), "\n", sayurB_siang.nama_sayur.to_string(index=False))
print("buah siang:\n", buah_siang.nama_buah.to_string(index=False), " ", buah_siang.porsi_urt.to_string(index=False))
print("\n\n\n################# CAMILAN SIANG ####################")
print(tabulate(tampil_camilan.loc[[1]], headers='keys', tablefmt='pretty'))
print("\n\n\n################# MAKAN MALAM ####################")
print(tabulate(tampil_malam, headers='keys', tablefmt='pretty'))
print("sayuran malam:\n", sayurA_malam.nama_sayur.to_string(index=False), "\n", sayurB_malam.nama_sayur.to_string(index=False))
print("buah malam:\n", buah_malam.nama_buah.to_string(index=False), " ", buah_malam.porsi_urt.to_string(index=False))
print("\n\n\n################# CAMILAN MALAM ####################")
print(tabulate(tampil_camilan.loc[[2]], headers='keys', tablefmt='pretty'))

: 